# VacationPy
----

In [1]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
vacation = pd.read_csv("output_file")
vacation.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Jamestown,90,US,1588752268,48,42.10,-79.24,39.20,9.17
1,1,Sibolga,100,ID,1588752342,67,1.74,98.78,86.02,3.98
2,2,Busselton,86,AU,1588752342,53,-33.65,115.33,57.00,17.65
3,3,Namatanai,100,PG,1588752343,71,-3.67,152.43,81.68,2.91
4,4,Cape Town,0,ZA,1588752343,72,-33.93,18.42,69.01,9.17


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Store Part I results into DataFrame

gmap = gmaps.configure(api_key=g_key)
locations = vacation[['Lat', 'Lng']]
weights = vacation['Humidity'] == 100
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps and humidity heatmap

hotel_df = vacation[((vacation["Max Temp"]>72) & (vacation["Max Temp"]<80) & (vacation["Wind Speed"]<10) & (vacation["Cloudiness"]==0))]
hotel_df = hotel_df.rename(columns={"Unnamed: 0": "Index"})
del hotel_df["Index"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
94,Bácum,0,MX,1588752262,34,27.55,-110.08,73.62,2.37
183,Oranjemund,0,NaN,1588751803,25,-28.55,16.43,74.05,5.68
428,Luxor,0,EG,1588752196,25,25.70,32.64,77.00,9.17
499,Bushenyi,0,UG,1588752362,73,-0.59,30.21,72.30,8.14


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Hotel Map and variable hotel_df
locations = []
params = {
    "types": "Hotel",
    "keyword": "Lodging",
    "radius": 5000,
    "key": g_key,
    }
dropped_rows = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    if 'status' in response and response['status']== "OK":
        locations.append(response['results'][0]['name'])
    else:
        dropped_rows.append(index)
hotel_df = hotel_df.drop(dropped_rows)
hotel_df["Hotel Name"] = locations

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
183,Oranjemund,0,NaN,1588751803,25,-28.55,16.43,74.05,5.68,Tom's cabin
428,Luxor,0,EG,1588752196,25,25.70,32.64,77.00,9.17,Hilton Luxor Resort & Spa
499,Bushenyi,0,UG,1588752362,73,-0.59,30.21,72.30,8.14,"Flagship Piazza Hotel, Bushenyi"


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))